In [2]:
import pandas as pd
import numpy as np
import os
import torch
import nltk

import networkx as nx
import matplotlib.pyplot as plt

In [3]:
data_dir = '../analysis_files/files/'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
# 감성대화말뭉치(최종데이터)_Training.csv 파일을 pandas로 읽어옵니다.
df = pd.read_csv(os.path.join(data_dir, '감성대화말뭉치(최종데이터)_Training.csv'), encoding='cp949')
df = df[['사람문장1', '시스템문장1', '사람문장2','시스템문장2','사람문장3','시스템문장3']]
df

,사람문장1,시스템문장1,사람문장2,시스템문장2,사람문장3,시스템문장3
0,일은 왜 해도 해도 끝이 없을까? 화가 난다.,많이 힘드시겠어요. 주위에 의논할 상대가 있나요?,그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고.,혼자 해결하기로 했군요. 혼자서 해결하기 힘들면 주위에 의논할 사람을 찾아보세요.,NaN,NaN
1,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?,최대한 지출을 억제해야겠어. 월급이 줄어들었으니 고정지출을 줄일 수밖에 없을 것 같아.,월급이 줄어든 만큼 소비를 줄일 계획이군요.,NaN,NaN
2,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 ...,잘 안 맞는 사람이랑 억지로 잘 지내는 것보단 조금은 거리를 두고 예의를 갖춰서 대...,스트레스받지 않기 위해선 인간관계에 있어 약간의 거리를 두는 게 좋겠군요.,NaN,NaN
3,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,관련 없는 심부름을 모두 하게 되어서 노여우시군요. 어떤 것이 상황을 나아질 수 있...,직장 사람들과 솔직하게 이야기해보고 싶어. 일하는 데에 방해된다고.,직장 사람들과 이야기를 해 보겠다고 결심하셨군요.,NaN,NaN
4,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,무시하는 것 같은 태도에 화가 나셨군요. 상대방의 어떤 행동이 그런 감정을 유발하는...,상사인 나에게 먼저 인사하지 않아서 매일 내가 먼저 인사한다고!,항상 먼저 인사하게 되어 화가 나셨군요. 어떻게 하면 신입사원에게 화났음을 표현할 ...,NaN,NaN
...,...,...,...,...,...,...
51625,나이가 먹고 이제 돈도 못 벌어 오니까 어떻게 살아가야 할지 막막해. 능력도 없고.,경제적인 문제 때문에 막막하시군요. 마음이 편치 않으시겠어요.,아무것도 할 수 없는 내가 무가치하게 느껴지고 실망스러워.,지금 할 수 있는 가장 합리적인 행동은 무엇인가요?,노년층을 위한 경제적 지원이나 부업 같은 것도 알아보아야겠어.,좋은 결과 얻으시길 바랄게요.
51626,몸이 많이 약해졌나 봐. 이제 전과 같이 일하지 못할 것 같아 너무 짜증 나.,건강에 대한 어려움 때문에 기분이 좋지 않으시군요. 속상하시겠어요.,마음 같아서는 다 할 수 있는 일인데 이젠 몸이 안 따라와 주니 화만 나.,어떻게 하면 지금의 기분을 나아지게 할 수 있을까요?,남편과 함께 게이트볼이나 치러 가야겠어. 그럼 기분이 나아질 것 같아.,남편과 함께하는 좋은 외출 시간 되시길 바랄게요.
51627,이제 어떻게 해야 할지 모르겠어. 남편도 그렇고 노후 준비도 안 되어서 미래가 걱정돼.,노후 준비에 대한 어려움 때문에 걱정이 많으시겠어요.,주변 사람들은 다 노후 준비도 잘해두었던데 난 어떻게 해야 할지 모르겠어. 막막하기...,지금의 상황에서 할 수 있는 가장 좋은 행동이 무엇일까요?,남편과 함께 실버 일자리나 노년층을 위한 국가 지원에 대해 자세히 알아보아야겠어.,좋은 정보 많이 얻으셔서 걱정을 좀 덜으셨으면 좋겠어요.
51628,몇십 년을 함께 살았던 남편과 이혼했어. 그동안의 세월에 배신감을 느끼고 너무 화가 나.,가족과의 문제 때문에 속상하시겠어요.,이제 할 수 있는 일도 없고 이렇게 힘들게 사는 게 불만스럽기만 해.,지금의 감정을 나아지게 할 수 있는 어떤 방법이 있을까요?,함께 친하게 지내던 동네 언니 동생들과 빈자리를 조금이나마 채울까 해.,지인분들과 좋은 시간 보내셨으면 좋겠어요.


In [5]:
# 그래프 생성
G = nx.Graph()

# 각 문장들을 conversation 데이터로 추출하고 그래프에 추가
for _, row in df.iterrows():
    for i in range(1, 4):
        user_sentence = row[f'사람문장{i}']
        system_sentence = row[f'시스템문장{i}']
        
        if pd.notnull(user_sentence):
            G.add_node(user_sentence, node_type='user')
        
        if pd.notnull(system_sentence):
            G.add_node(system_sentence, node_type='system')
        
        if pd.notnull(user_sentence) and pd.notnull(system_sentence):
            G.add_edge(user_sentence, system_sentence)

In [6]:
# 그래프 정보 출력 (일부)
num_nodes_to_print = 5
num_edges_to_print = 5

print('노드 개수:', G.number_of_nodes())
print('엣지 개수:', G.number_of_edges())

# 상위 n개의 노드 출력
print(f'\n상위 {num_nodes_to_print}개의 노드:')
for node in list(G.nodes)[:num_nodes_to_print]:
    print(node)

# 상위 n개의 엣지 출력
print(f'\n상위 {num_edges_to_print}개의 엣지:')
for edge in list(G.edges)[:num_edges_to_print]:
    print(edge)

노드 개수: 284033
엣지 개수: 145351

상위 5개의 노드:
일은 왜 해도 해도 끝이 없을까? 화가 난다.
많이 힘드시겠어요. 주위에 의논할 상대가 있나요?
그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고.
혼자 해결하기로 했군요. 혼자서 해결하기 힘들면 주위에 의논할 사람을 찾아보세요. 
이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.

상위 5개의 엣지:
('일은 왜 해도 해도 끝이 없을까? 화가 난다.', '많이 힘드시겠어요. 주위에 의논할 상대가 있나요?')
('그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고.', '혼자 해결하기로 했군요. 혼자서 해결하기 힘들면 주위에 의논할 사람을 찾아보세요. ')
('이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.', '급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?')
('최대한 지출을 억제해야겠어. 월급이 줄어들었으니 고정지출을 줄일 수밖에 없을 것 같아.', '월급이 줄어든 만큼 소비를 줄일 계획이군요.')
('회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스트레스 받아. ', '회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 하면 좋을까요?')


In [7]:
# 그래프 시각화
pos = nx.spring_layout(G)  # 그래프 레이아웃 설정
node_colors = {'user': 'blue', 'system': 'red'}  # 노드 색상 설정

plt.figure(figsize=(10, 6))  # 그래프 크기 설정

# 노드 그리기
for node, node_type in nx.get_node_attributes(G, 'node_type').items():
    nx.draw_networkx_nodes(G, pos, nodelist=[node], node_color=node_colors[node_type], node_size=500, alpha=0.8)

# 엣지 그리기
nx.draw_networkx_edges(G, pos, width=1.0, alpha=0.5)

# 노드 라벨 그리기
nx.draw_networkx_labels(G, pos, font_size=10, font_color='black')

plt.axis('off')  # 축 숨김
plt.title('Conversation Graph')  # 그래프 제목
plt.show()

KeyboardInterrupt: 